In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
  llm=llm,
  max_token_limit=150,
  return_messages=True
)

In [2]:
examples = [
  {
    "movie": "탑건",
    "answer": "🛩️👨‍✈️🔥",
  },
  {
    "movie": "대부",
    "answer": "👨‍👨‍👦🔫🍝"
  }
]

example_prompt = ChatPromptTemplate.from_messages(
  [
    ("human", "{movie}"),
    ("ai", "{answer}")
  ]
)

few_prompt = FewShotChatMessagePromptTemplate(
  examples=examples,
  example_prompt=example_prompt
)

prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "You have the ability to describe a movie in three images."),
    MessagesPlaceholder(variable_name="history"),
    few_prompt,
    ("human", "{movie}")
  ]
)

In [3]:
def load_memory(movie):
  return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(movie):
  result = chain.invoke({"movie": movie})
  memory.save_context(
    {"input": movie},
    {"output": result.content},
  )
  print(result)

In [4]:
invoke_chain("아바타")

content='🌿🌊🔵'


In [5]:
invoke_chain("탑건")

content='🛩️🕶️🔥'


In [6]:
invoke_chain("먼저 질문한 영화가 뭐였지?")

content="먼저 질문한 영화는 '탑건'이었습니다."
